## Repliikide eraldamine ja andmebaasi loomine test5

In [1]:
from estnltk import Text
import re
import regex
from estnltk.taggers.system.rule_taggers import RegexTagger, Ruleset
from estnltk.taggers.system.rule_taggers import StaticExtractionRule
from estnltk.taggers import HeaderBasedSegmenter
from estnltk.taggers import NerTagger
import sqlite3
import os
import datetime
import pickle

Kõnelejale vastav regulaaravaldis:

In [2]:
pattern = r'((^|\n)(?P<span>(?P<name>.*): ?))'

Funktsioon kõneleja nime atribuudi lisamiseks:

In [3]:
def decorator(layer,span,annotation):
    annotation['person'] = annotation['match'].group(4)
    return annotation

Lisatakse reegel kõneleja fikseeriva tekstifragmendi väljavõtmiseks:

In [4]:
rule = [StaticExtractionRule(pattern=regex.Regex('((^|\n)(?P<span>(?P<name>.*): ?))'), attributes={'_group_': 0}, group=3)]
ruleset = Ruleset()
ruleset.add_rules(rule)

Märgendaja RegexTagger loomine:

In [5]:
tagger = RegexTagger(
    ruleset = ruleset,
    output_layer = 'headers',
    output_attributes = ['person'],
    decorator=decorator,
    match_attribute='match'
) 

Märgendaja HeaderBasedSegmenter loomine:

In [6]:
tagger2 = HeaderBasedSegmenter(
    input_layer = 'headers', 
    output_layer = 'lines',
    output_attributes = ['person'],
    decorator = lambda header_span: {'person': header_span['person'][0]},
    include_header = False
)

Nimeüksuste märgendaja NerTagger:

In [7]:
nertagger = NerTagger()

Luuakse andmebaas ja seal uus tabel, kuhu lisatakse kõik repliikidele vastavad Text-objektid, samuti info kuupäeva, kõneleja nime ja repliigi numbri kohta.

In [8]:
con = sqlite3.connect("data.db")
cur = con.cursor()
cur.execute('pragma encoding=UTF8')
cur.execute("CREATE TABLE data(line BLOB, number INTEGER, person TEXT, date DATE)")

In [11]:
path = 'test5/'
for file in os.listdir(path):
    with open(os.path.join(path, file), mode="r", encoding="utf-8") as f:
        raw_text = f.read()
        filename = file.replace('.txt', '')
        date = datetime.datetime.strptime(filename, "%Y-%m-%d").date()
        
        text = tagger.tag(Text(raw_text))
        text = tagger2.tag(text)
        
        for i in range(len(text.lines)):
            span = text.lines[i]
            line = span.text.strip()
            if line == '':
                continue
            number = i+1
            line = Text(line)
            line.meta['line_number'] = number
            line.meta['person'] = span.person
            line.meta['date'] = date
            line.tag_layer(['sentences', 'morph_analysis'])
            nertagger.tag(line)
            line_bytes = pickle.dumps(line)
            cur.execute("""INSERT INTO data
                                  (line, number, person, date) 
                                  VALUES (?, ?, ?, ?);""", ((bytearray(line_bytes)), number, span.person, date))
            con.commit()

In [12]:
con.close()